In [1]:
setwd('/nfs/lab/projects/islet_cytok/analysis/snATAC/coaccess')

In [2]:
suppressPackageStartupMessages(library(data.table))
suppressPackageStartupMessages(library(stringr))
suppressPackageStartupMessages(library(RColorBrewer))

In [3]:
cells = c('A471.EndoC_cyt','A471.EndoC_unt')

In [18]:
### remove scientific notation form the coordinates
for (cell in cells){
file = paste0(cell,  '.FDR10.bedpe')
test = fread(file)
    fwrite(test, file, sep="\t", quote=F, row.names=F, col.names = F,scipen = 999)
    }


In [20]:
for (cell in cells){
file     = paste0(cell,  '.FDR10.bedpe')
filesort = paste0(cell,  '.FDR10.bedpe.sorted')
prom     = "/nfs/lab/publicdata/gencode_v19/gencode.v19.5kb_all_possible_transcripts.bed"
out      = paste0(cell,  '.FDR10.bedpe.promLabelled')
system (paste("pgltools sort", file, ">", filesort))
system (paste('pgltools intersect1D -a' , filesort , '-b', prom , '-wa -allA >' ,out))
}

In [66]:
for (cell in cells){
file     = paste0(cell,  '.FDR10.bedpe.promLabelled')
fileout  = paste0(cell,  '.FDR10.bedpe.promLabelled.dedup.cp')

test = read.table(file)
test =test[!duplicated(test),]
test$V19 = apply(test[,1:7],1, paste, collapse="_")
ag = aggregate(V9~V19, test, paste, collapse=",")
test = merge(test, ag, by="V19")
test =test[!duplicated(test[,c(1,11)]),-10]
test$V20 = apply(test[,2:7],1, paste, collapse="_")
ag = aggregate(V7~V20, test, paste, collapse=",")

#table(ag$V7)
ag = subset(ag, V7!="A,B")
test = subset(test, V20 %in% ag$V20)
test$V9 = test$V9.y
test$V9[test$V7=="B"]<-"."
test$V10 = test$V9.y
test$V10[test$V7=="A"]<-"."
test$V8 = "CP"
subtest = test[,c('V1','V2','V3','V4','V5','V6','V8','V9','V10')]
fwrite(subtest, fileout, sep="\t", quote=F, row.names=F, col.names = F,scipen = 999)
  }

In [67]:
for(cell in cells){
system(paste('pgltools intersect1D -a', paste0(cell,  '.FDR10.bedpe.promLabelled.dedup.cp') ,  
             '-b ../../ldsc/Islet_bulkATAC.bed -allA -wa -wb >',
       paste0(cell, ".pchich.promlab.fdr10.intersect.alsites")))
}